# Sentiment Analysis App with Huggingface Distributed GPU Training and PyTorch Lightning

In this notebook, we will reimplement the work done in the SageMaker Project notebook, but with a language model obtained from Huggingface and using PyTorch Lightning to train the model. We are doing this because the model trained in the SageMaker Notebook used an LSTM, which is quite outdated for language models.

## Setup

In [ ]:
!pip install -q transformers git+https://github.com/huggingface/datasets.git --upgrade
!pip install -q torch 
!pip install pytorch-lightning --quiet
!pip install wandb --quiet